In [184]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import resample

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [185]:
# Carregar os arquivos CSV
base_cadastral = pd.read_csv('base_cadastral.csv')
base_info = pd.read_csv('base_info.csv')
base_pagamentos = pd.read_csv('base_pagamentos_desenvolvimento.csv')

In [186]:
# Junção das bases de dados
merged_data = pd.merge(base_cadastral, base_info, on='ID_CLIENTE', how = 'inner')
merged_data = pd.merge(merged_data, base_pagamentos, on=['ID_CLIENTE', 'SAFRA_REF'], how = 'inner')
merged_data.head()

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA
0,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,2018-09,16913.00,92.00,2018-09-16,2018-10-09,2018-10-08,22427.25,5.99
1,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,2018-09,16913.00,92.00,2018-09-23,2018-10-15,2018-10-15,35608.11,5.99
2,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,2018-10,236447.00,93.00,2018-10-08,2018-10-30,2018-10-29,17988.49,5.99
3,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,2018-10,236447.00,93.00,2018-10-17,2018-11-07,2018-11-06,41998.20,6.99
4,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,2018-10,236447.00,93.00,2018-10-21,2018-11-12,2018-11-12,35514.41,6.99


In [187]:
#Análise exploratoria
merged_data.info()
# merged_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73477 entries, 0 to 73476
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_CLIENTE              73477 non-null  int64  
 1   DATA_CADASTRO           73477 non-null  object 
 2   DDD                     66574 non-null  object 
 3   FLAG_PF                 156 non-null    object 
 4   SEGMENTO_INDUSTRIAL     72183 non-null  object 
 5   DOMINIO_EMAIL           72656 non-null  object 
 6   PORTE                   71112 non-null  object 
 7   CEP_2_DIG               73477 non-null  object 
 8   SAFRA_REF               73477 non-null  object 
 9   RENDA_MES_ANTERIOR      73477 non-null  float64
 10  NO_FUNCIONARIOS         73477 non-null  float64
 11  DATA_EMISSAO_DOCUMENTO  73477 non-null  object 
 12  DATA_PAGAMENTO          73477 non-null  object 
 13  DATA_VENCIMENTO         73477 non-null  object 
 14  VALOR_A_PAGAR           73477 non-null

In [188]:
merged_data = merged_data.drop_duplicates(subset=['ID_CLIENTE', 'SAFRA_REF'])
merged_data

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA
0,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,2018-09,16913.00,92.00,2018-09-16,2018-10-09,2018-10-08,22427.25,5.99
2,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,2018-10,236447.00,93.00,2018-10-08,2018-10-30,2018-10-29,17988.49,5.99
9,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,2018-11,107840.00,91.00,2018-11-01,2018-12-10,2018-12-05,1341.00,6.99
13,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,2018-12,164029.00,87.00,2018-12-02,2018-12-24,2018-12-24,16869.11,4.99
16,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,2019-01,128674.00,96.00,2019-01-11,2019-01-31,2019-01-31,19173.70,6.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73471,8736788139020105764,2021-02-04,NaN,NaN,Serviços,HOTMAIL,MEDIO,13,2021-06,62176.00,129.00,2021-06-13,2021-09-27,2021-09-01,48.00,4.99
73472,3148228961195376491,2015-12-13,NaN,NaN,Comércio,BOL,MEDIO,36,2021-06,343507.00,134.00,2021-06-13,2021-07-26,2021-07-28,17302.05,4.99
73473,2439954154030259398,2019-07-02,NaN,NaN,Indústria,BOL,PEQUENO,13,2021-06,392975.00,121.00,2021-06-14,2021-06-28,2021-06-29,19799.30,6.99
73475,4799794458216343260,2019-12-30,19,NaN,Indústria,OUTLOOK,MEDIO,20,2021-06,70449.00,141.00,2021-06-14,2021-07-13,2021-07-09,2806.09,6.99


In [189]:
for col in merged_data.columns:
    print(f'{col}:\n{merged_data[col].unique()}\n')

ID_CLIENTE:
[1661240395903230676 8274986328479596038  345447888460137901 ...
 2439954154030259398 4799794458216343260 1246864346193559124]

DATA_CADASTRO:
['2013-08-22' '2017-01-25' '2000-08-15' '2017-08-06' '2011-02-14'
 '2018-03-27' '2009-08-30' '2014-06-23' '2001-05-31' '2016-02-24'
 '2006-07-18' '2016-08-14' '2006-09-11' '2012-05-29' '2014-03-02'
 '2017-03-09' '2015-11-23' '2012-03-11' '2003-06-03' '2016-11-29'
 '2017-08-09' '2013-07-29' '2015-11-25' '2002-01-24' '2017-10-19'
 '2014-03-13' '2007-06-17' '2011-02-15' '2012-08-27' '2014-08-14'
 '2015-03-02' '2015-12-29' '2015-09-14' '2005-11-24' '2014-08-07'
 '2012-12-13' '2015-12-08' '2007-02-05' '2011-12-22' '2015-02-27'
 '2016-10-30' '2008-06-03' '2007-07-31' '2009-08-18' '2014-07-22'
 '2010-08-04' '2009-03-30' '2007-10-17' '2007-03-19' '2013-10-21'
 '2016-10-04' '2015-04-01' '2014-02-06' '2011-07-13' '2009-08-02'
 '2015-11-24' '2012-12-06' '2011-05-16' '2008-05-22' '2017-02-21'
 '2015-06-23' '2018-03-20' '2014-07-23' '2011-05-19' 

In [190]:
# Conversões de tipos de variaveis
merged_data = merged_data.replace('na', np.nan)

date_cols = ['DATA_CADASTRO', 
             'SAFRA_REF', 
             'DATA_EMISSAO_DOCUMENTO', 
             'DATA_PAGAMENTO', 
             'DATA_VENCIMENTO']

for col in date_cols:
    merged_data[col] = pd.to_datetime(merged_data[col])

# merged_data['DATA_CADASTRO'] = pd.to_datetime(merged_data['DATA_CADASTRO']).dt.date
# merged_data['SAFRA_REF'] = pd.to_datetime(merged_data['SAFRA_REF']).dt.date
# merged_data['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(merged_data['DATA_EMISSAO_DOCUMENTO']).dt.date
# merged_data['DATA_PAGAMENTO'] = pd.to_datetime(merged_data['DATA_PAGAMENTO']).dt.date
# merged_data['DATA_VENCIMENTO'] = pd.to_datetime(merged_data['DATA_VENCIMENTO']).dt.date
merged_data['DDD'] = merged_data['DDD'].str.replace(r'\(|\)', '', regex=True)

merged_data.fillna({'DDD': 0, 
                    'FLAG_PF': 'Y', 
                    'SEGMENTO_INDUSTRIAL': 'Outros',
                    'DOMINIO_EMAIL': 'NAO INFORMADO',
                    'PORTE': 'NAO INFORMADO', 
                    'CEP_2_DIG' : 0}, 
                   inplace=True)
                   
for col in ['DDD', 'CEP_2_DIG', 'NO_FUNCIONARIOS']:
    merged_data[col] = merged_data[col].astype(int)

In [191]:
# Análise exploratoria
merged_data.info()
merged_data.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 18552 entries, 0 to 73476
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID_CLIENTE              18552 non-null  int64         
 1   DATA_CADASTRO           18552 non-null  datetime64[ns]
 2   DDD                     18552 non-null  int64         
 3   FLAG_PF                 18552 non-null  object        
 4   SEGMENTO_INDUSTRIAL     18552 non-null  object        
 5   DOMINIO_EMAIL           18552 non-null  object        
 6   PORTE                   18552 non-null  object        
 7   CEP_2_DIG               18552 non-null  int64         
 8   SAFRA_REF               18552 non-null  datetime64[ns]
 9   RENDA_MES_ANTERIOR      18552 non-null  float64       
 10  NO_FUNCIONARIOS         18552 non-null  int64         
 11  DATA_EMISSAO_DOCUMENTO  18552 non-null  datetime64[ns]
 12  DATA_PAGAMENTO          18552 non-null  datetime64[

,ID_CLIENTE,DATA_CADASTRO,DDD,CEP_2_DIG,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA
count,18552.00,18552,18552.00,18552.00,18552,18552.00,18552.00,18552,18552,18552,18552.00,18552.00
mean,4671549994484296704.00,2010-12-07 09:50:31.824062208,37.28,53.28,2020-02-02 17:09:42.147477248,289694.95,116.89,2020-02-09 12:40:45.019405056,2020-03-03 19:14:26.235446272,2020-03-03 18:31:40.129366272,44570.06,6.80
min,8784237149961904.00,2000-08-15 00:00:00,0.00,0.00,2018-09-01 00:00:00,105.00,0.00,2018-09-01 00:00:00,2018-06-19 00:00:00,2017-11-27 00:00:00,11.70,4.99
25%,2347028805684088320.00,2007-03-19 00:00:00,12.00,28.00,2019-06-01 00:00:00,133561.00,105.00,2019-06-02 00:00:00,2019-06-21 00:00:00,2019-06-21 00:00:00,17988.40,5.99
50%,4797473719871088640.00,2011-02-15 00:00:00,32.00,55.00,2020-02-01 00:00:00,240824.00,117.00,2020-02-09 00:00:00,2020-03-05 00:00:00,2020-03-05 00:00:00,34291.40,5.99
75%,7070921952745705472.00,2015-08-16 00:00:00,61.00,79.00,2020-11-01 00:00:00,394619.75,129.00,2020-11-02 00:00:00,2020-11-23 00:00:00,2020-11-23 00:00:00,58985.70,6.99
max,9206030810342980608.00,2021-06-01 00:00:00,99.00,99.00,2021-06-01 00:00:00,1682759.00,198.00,2021-06-30 00:00:00,2021-10-20 00:00:00,2025-08-27 00:00:00,1500000.00,11.99
std,2671111043542919168.00,NaN,28.32,28.37,NaN,212461.16,20.33,NaN,NaN,NaN,41296.66,1.82


In [192]:
# Calculando a diferença entre as datas de pagamento e vencimento
diferenca = (merged_data['DATA_PAGAMENTO'] - merged_data['DATA_VENCIMENTO']).dt.days

# Criando a coluna de inadimplência
merged_data['INADIMPLENCIA'] = (diferenca >= 5).astype(int)

In [193]:
merged_data['INADIMPLENCIA'].value_counts() # base desbalanceada

INADIMPLENCIA
0    17404
1     1148
Name: count, dtype: int64

In [194]:
# Balanceamento da amostra
# Separe as classes majoritária e minoritária
df_majority = merged_data[merged_data['INADIMPLENCIA'] == 0]
df_minority = merged_data[merged_data['INADIMPLENCIA'] == 1]

# Subamostre a classe majoritária
df_majority_undersampled = resample(df_majority,
                                    replace=False,
                                    n_samples=len(df_minority),
                                    random_state=0)

# Combine a classe majoritária subamostrada com a classe minoritária
df_undersampled = pd.concat([df_majority_undersampled, df_minority])

# Exiba as novas contagens de classes
print(df_undersampled['INADIMPLENCIA'].value_counts())

INADIMPLENCIA
0    1148
1    1148
Name: count, dtype: int64


In [195]:
df_undersampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2296 entries, 31755 to 73471
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID_CLIENTE              2296 non-null   int64         
 1   DATA_CADASTRO           2296 non-null   datetime64[ns]
 2   DDD                     2296 non-null   int64         
 3   FLAG_PF                 2296 non-null   object        
 4   SEGMENTO_INDUSTRIAL     2296 non-null   object        
 5   DOMINIO_EMAIL           2296 non-null   object        
 6   PORTE                   2296 non-null   object        
 7   CEP_2_DIG               2296 non-null   int64         
 8   SAFRA_REF               2296 non-null   datetime64[ns]
 9   RENDA_MES_ANTERIOR      2296 non-null   float64       
 10  NO_FUNCIONARIOS         2296 non-null   int64         
 11  DATA_EMISSAO_DOCUMENTO  2296 non-null   datetime64[ns]
 12  DATA_PAGAMENTO          2296 non-null   datetime

In [196]:
# Codificar variáveis categóricas
df_undersampled = pd.get_dummies(df_undersampled, columns=['FLAG_PF', 
                                                           'SEGMENTO_INDUSTRIAL', 
                                                           'DOMINIO_EMAIL', 
                                                           'PORTE'])

In [197]:
# Divisão dos dados em conjunto de treinamento e conjunto de teste/validação
X = df_undersampled.drop(['ID_CLIENTE', 
                          'DATA_CADASTRO',
                          'SAFRA_REF', 
                          'DATA_EMISSAO_DOCUMENTO',
                          'DATA_PAGAMENTO',
                          'DATA_VENCIMENTO',
                          'INADIMPLENCIA'], axis=1)
y = df_undersampled['INADIMPLENCIA']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [198]:
# Treinamento e avaliação dos modelos
models = [
    LogisticRegression(),
    RandomForestClassifier(),
    XGBClassifier()
]

metrics = {
    'AUC-ROC': roc_auc_score,
    'Accuracy': accuracy_score,
    'Precision': precision_score,
    'Recall': recall_score,
    'F1-score': f1_score
}

for model in models:
    model_name = model.__class__.__name__
    print(f"Training and evaluating {model_name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    for metric_name, metric_func in metrics.items():
        score = metric_func(y_test, y_pred)
        print(f"{metric_name}: {score:.4f}")
    
    print("--------------------")


Training and evaluating LogisticRegression...
AUC-ROC: 0.6746
Accuracy: 0.6761
Precision: 0.6827
Recall: 0.7083
F1-score: 0.6953
--------------------
Training and evaluating RandomForestClassifier...
AUC-ROC: 0.8263
Accuracy: 0.8239
Precision: 0.8768
Recall: 0.7708
F1-score: 0.8204
--------------------
Training and evaluating XGBClassifier...
AUC-ROC: 0.7996
Accuracy: 0.7978
Precision: 0.8387
Recall: 0.7583
F1-score: 0.7965
--------------------


In [199]:
# Treinamento do modelo final
final_model = RandomForestClassifier()
final_model.fit(X, y)

# Previsão dos dados de teste
base_pagamentos_teste = pd.read_csv('base_pagamentos_teste.csv')
merged_test_data = pd.merge(base_cadastral, base_info, on='ID_CLIENTE', how = 'inner')
merged_test_data = pd.merge(merged_test_data, base_pagamentos_teste, on=['ID_CLIENTE', 'SAFRA_REF'], how = 'inner')

merged_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11830 entries, 0 to 11829
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_CLIENTE              11830 non-null  int64  
 1   DATA_CADASTRO           11830 non-null  object 
 2   DDD                     10562 non-null  object 
 3   FLAG_PF                 34 non-null     object 
 4   SEGMENTO_INDUSTRIAL     11634 non-null  object 
 5   DOMINIO_EMAIL           11666 non-null  object 
 6   PORTE                   11461 non-null  object 
 7   CEP_2_DIG               11830 non-null  object 
 8   SAFRA_REF               11830 non-null  object 
 9   RENDA_MES_ANTERIOR      11830 non-null  float64
 10  NO_FUNCIONARIOS         11830 non-null  float64
 11  DATA_EMISSAO_DOCUMENTO  11830 non-null  object 
 12  DATA_VENCIMENTO         11830 non-null  object 
 13  VALOR_A_PAGAR           11830 non-null  float64
 14  TAXA                    11830 non-null

In [ ]:
for col in merged_test_data.columns:
    print(f'{col}:\n{merged_test_data[col].unique()}\n')

In [201]:
# Conversões de tipos de variaveis
merged_test_data = merged_test_data.replace('na', np.nan)

date_cols = ['DATA_CADASTRO', 
             'SAFRA_REF', 
             'DATA_EMISSAO_DOCUMENTO', 
             'DATA_VENCIMENTO']

for col in date_cols:
    merged_test_data[col] = pd.to_datetime(merged_test_data[col])

# merged_data['DATA_CADASTRO'] = pd.to_datetime(merged_data['DATA_CADASTRO']).dt.date
# merged_data['SAFRA_REF'] = pd.to_datetime(merged_data['SAFRA_REF']).dt.date
# merged_data['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(merged_data['DATA_EMISSAO_DOCUMENTO']).dt.date
# merged_data['DATA_PAGAMENTO'] = pd.to_datetime(merged_data['DATA_PAGAMENTO']).dt.date
# merged_data['DATA_VENCIMENTO'] = pd.to_datetime(merged_data['DATA_VENCIMENTO']).dt.date
merged_test_data['DDD'] = merged_test_data['DDD'].str.replace(r'\(|\)', '', regex=True)

merged_test_data.fillna({'DDD': 0, 
                    'FLAG_PF': 'Y', 
                    'SEGMENTO_INDUSTRIAL': 'Outros',
                    'DOMINIO_EMAIL': 'NAO INFORMADO',
                    'PORTE': 'NAO INFORMADO', 
                    'CEP_2_DIG' : 0}, 
                   inplace=True)
                   
for col in ['DDD', 'CEP_2_DIG', 'NO_FUNCIONARIOS']:
    merged_test_data[col] = merged_test_data[col].astype(int)

In [202]:
merged_test_data = pd.get_dummies(merged_test_data, columns=['FLAG_PF', 
                                                           'SEGMENTO_INDUSTRIAL', 
                                                           'DOMINIO_EMAIL', 
                                                           'PORTE'])

In [ ]:
X_test_final = merged_test_data.drop(['ID_CLIENTE', 
                          'DATA_CADASTRO',
                          'SAFRA_REF', 
                          'DATA_EMISSAO_DOCUMENTO',
                          'DATA_VENCIMENTO'], axis=1)

predictions = final_model.predict_proba(X_test_final)[:, 1]

# Criar a base de dados de teste com as colunas ID_CLIENTE, SAFRA_REF e INADIMPLENTE
test_data = merged_test_data[['ID_CLIENTE', 'SAFRA_REF']]
test_data['INADIMPLENTE'] = predictions
test_data.to_csv('previsoes_teste.csv', index=False)